In [1]:
'''
Patrick Ballou
ID: 801130521
ECGR 4106
Homework 5
Problem 4
'''

'\nPatrick Ballou\nID: 801130521\nECGR 4106\nHomework 5\nProblem 4\n'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch import cuda
from torch.utils.data import DataLoader, Dataset

In [ ]:
#check if GPU is available and set the device accordingly
#device = 'torch.device("cuda:0" if torch.cuda.is_available() else "cpu")'
device = 'cuda'
print("Using GPU: ", cuda.get_device_name())

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)